none of the dir or mngr of admin are connected to other departments
sales -> manager 1, 2 and 3 are connected to the ceos but also the directors

In [1]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import HTML

In [2]:
data = pd.read_csv("data/edgelist.csv", index_col=0, encoding = "utf-8")
data

,Person 1,Person 2
1,Admin.mgr1,Admin.2
2,Sales.mgr3,Sales.8
3,Production.2,Production.3
4,Production.2,Production.7
5,Sales.1,Sales.2
...,...,...
136,Production.mgr1,Production.2
137,Sales.Dir,Sales.6
138,Admin.4,Production.11
139,Admin.mgr1,Admin.3


In [3]:
data["weight person 1"] = 1
data["weight person 1"] = data.apply(lambda x: 1.2 if x["Person 1"].split(".")[-1][0] == "m" 
                                     else (1.8 if x["Person 1"].split(".")[-1][0] == "D" else 1) , axis = 1)
data["weight person 2"] = 1
data["weight person 2"] = data.apply(lambda x: 1.2 if x["Person 2"].split(".")[-1][0] == "m" 
                                     else (1.8 if x["Person 2"].split(".")[-1][0] == "D" else 1) , axis = 1)
data["weight person 1"] = data["weight person 1"].where(data["Person 1"]!="CEO", 3)
data["weight person 2"] = data["weight person 2"].where(data["Person 2"]!="CEO", 3)
data["weight"] = data["weight person 1"] * data["weight person 2"]
data

,Person 1,Person 2,weight person 1,weight person 2,weight
1,Admin.mgr1,Admin.2,1.2,1.0,1.2
2,Sales.mgr3,Sales.8,1.2,1.0,1.2
3,Production.2,Production.3,1.0,1.0,1.0
4,Production.2,Production.7,1.0,1.0,1.0
5,Sales.1,Sales.2,1.0,1.0,1.0
...,...,...,...,...,...
136,Production.mgr1,Production.2,1.2,1.0,1.2
137,Sales.Dir,Sales.6,1.8,1.0,1.8
138,Admin.4,Production.11,1.0,1.0,1.0
139,Admin.mgr1,Admin.3,1.2,1.0,1.2


In [4]:
G = nx.from_pandas_edgelist(data, source="Person 1", target="Person 2")

In [5]:
for u, v in G.edges():
    if data[(data["Person 1"]==u)&(data["Person 2"]==v)]["weight"].empty:
        weight = float(data[(data["Person 2"]==u)&(data["Person 1"]==v)]["weight"])
    else:
        weight = float(data[(data["Person 1"]==u)&(data["Person 2"]==v)]["weight"])
    G[u][v]["weight"] = weight

C:\Users\teuta\AppData\Local\Temp\ipykernel_16576\207571619.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  weight = float(data[(data["Person 1"]==u)&(data["Person 2"]==v)]["weight"])
C:\Users\teuta\AppData\Local\Temp\ipykernel_16576\207571619.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  weight = float(data[(data["Person 2"]==u)&(data["Person 1"]==v)]["weight"])


In [6]:
centrality = nx.eigenvector_centrality(G, weight='weight')
centrality
centrality_sorted = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
centrality_sorted

[('CEO', 0.5388181797222633),
 ('Sales.Dir', 0.4185926720900137),
 ('Sales.mgr1', 0.2878301332612606),
 ('Sales.mgr3', 0.277166343942355),
 ('Sales.mgr2', 0.2699315805152943),
 ('Production.Dir', 0.22219880194355096),
 ('Admin.Dir', 0.2142576383521509),
 ('Production.mgr1', 0.1675111260532105),
 ('Admin.mgr1', 0.1534043096822214),
 ('Sales.8', 0.14920029727822626),
 ('Sales.1', 0.12677065996808334),
 ('Sales.6', 0.12406700102858895),
 ('Sales.9', 0.12334826309704736),
 ('Sales.3', 0.11743966642231275),
 ('Sales.4', 0.11665421954289949),
 ('Sales.10', 0.11051303622433599),
 ('Sales.5', 0.10355290294478606),
 ('Sales.2', 0.10232982791724853),
 ('Sales.7', 0.09645630369838955),
 ('Production.mgr2', 0.051085000156848165),
 ('Production.mgr3', 0.051012364597909206),
 ('Admin.2', 0.03760885798591363),
 ('Production.11', 0.03432856584457812),
 ('Production.mgr4', 0.03422291033186421),
 ('Admin.mgr2', 0.03218043856978444),
 ('Production.1', 0.030061414271645347),
 ('Admin.mgr3', 0.028916805879

In [7]:
betweenness = nx.betweenness_centrality(G)
betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)

In [8]:
from sklearn.preprocessing import MinMaxScaler
centrality_df = pd.DataFrame(centrality_sorted, columns=["Employee", "rank"])
scaler = MinMaxScaler()
centrality_df["centrality_rank"] = scaler.fit_transform(centrality_df["rank"].array.reshape(-1, 1))
df = centrality_df.drop("rank", axis=1)
df

,Employee,centrality_rank
0,CEO,1.000000
1,Sales.Dir,0.775986
2,Sales.mgr1,0.532339
3,Sales.mgr3,0.512470
4,Sales.mgr2,0.498989
5,Production.Dir,0.410050
6,Admin.Dir,0.395253
7,Production.mgr1,0.308151
8,Admin.mgr1,0.281866
9,Sales.8,0.274033


In [9]:
from sklearn.preprocessing import MinMaxScaler
betweenness_df = pd.DataFrame(betweenness, columns=["Employee", "betweennes"])
scaler = MinMaxScaler()
#centrality_df["rank"] = 
betweenness_df["betweennes"] = scaler.fit_transform(betweenness_df["betweennes"].array.reshape(-1, 1))
betweenness_df

,Employee,betweennes
0,CEO,1.000000
1,Production.mgr1,0.783388
2,Admin.4,0.449441
3,Admin.Dir,0.342256
4,Sales.1,0.335216
5,Production.11,0.319525
6,Sales.mgr1,0.290588
7,Sales.Dir,0.255296
8,Production.7,0.250194
9,Production.Dir,0.236851


In [10]:
df_tot = betweenness_df.merge(df, how="left", on="Employee")
df_tot["rank"] = (df_tot["betweennes"] + df_tot["centrality_rank"])/2
df_tot.sort_values("rank", ascending=False, inplace=True)

In [11]:
import numpy as np
c = np.arange(0, 0xFFFFFF, 0xFFFFFF/46)
df_tot["color"] = c
df_tot

,Employee,betweennes,centrality_rank,rank,color
0,CEO,1.000000,1.000000,1.000000,0.000000e+00
1,Production.mgr1,0.783388,0.308151,0.545770,3.647221e+05
7,Sales.Dir,0.255296,0.775986,0.515641,7.294441e+05
6,Sales.mgr1,0.290588,0.532339,0.411464,1.094166e+06
3,Admin.Dir,0.342256,0.395253,0.368755,1.458888e+06
13,Sales.mgr3,0.137157,0.512470,0.324813,1.823610e+06
9,Production.Dir,0.236851,0.410050,0.323450,2.188332e+06
16,Sales.mgr2,0.108255,0.498989,0.303622,2.553054e+06
4,Sales.1,0.335216,0.232240,0.283728,2.917777e+06
2,Admin.4,0.449441,0.025296,0.237368,3.282499e+06


In [12]:
import plotly.express as px
import numpy as np
c = np.arange(0, 0xFFFFFF, 0xFFFFFF/46)
px.bar(df_tot, x = "Employee", y="rank")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Employee=%{x}<br>rank=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['CEO', 'Production.mgr1', 'Sales.Dir', 'Sales.mgr1', 'Admin.Dir',
                          'Sales.mgr3', 'Production.Dir', 'Sales.mgr2', 'Sales.1', 'Admin.4',
                          'Admin.mgr1', 'Production.11', 'Sales.8', 'Production.7', 'Sales.9',
                          'Sales.6', 'Production.14', 'Admin.1', 'Sales.3', 'Sales.4', 'Sales.2',
                          'Sales.10', 'Production.mgr2', 'Admin.mgr3', 'Production.mgr3',
                          'Sales.5', 'Sales.7', 'Admin.6', 'Production.10', 'Production.mgr4',
                          'Admin.5', 'Production.1', 'Admin.mgr2', 'Production.3', 'Admin.2',
                          'Admin.3', 'Production.9', 'Admin.8', 'Production.2', 'Production.4',
                          'Production.12', 'Production.13', 'Production.5', 'Production.6',
                          'Production.8', 'Admin.7'], dtype=object),
              'xaxis': 'x',
              'y': array([1.        , 0.5457697 , 0.51564115, 0.41146369, 0.36875463, 0.32481332,
                          0.32345043, 0.30362213, 0.28372824, 0.23736835, 0.19602782, 0.18975999,
                          0.14556711, 0.14234309, 0.12276417, 0.12046323, 0.11388848, 0.11366544,
                          0.11341419, 0.11187892, 0.10321701, 0.10240735, 0.09882325, 0.09831539,
                          0.09826167, 0.0966784 , 0.08949368, 0.08704002, 0.04938315, 0.04910161,
                          0.04865225, 0.04692031, 0.04186455, 0.03877852, 0.03605007, 0.03273715,
                          0.02148588, 0.01868671, 0.01853859, 0.01423452, 0.00371227, 0.00371227,
                          0.00300813, 0.00300813, 0.00294561, 0.        ]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Employee'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'rank'}}}
})

In [ ]:
from colour import Color
start = Color("red")
colors = list(start.range_to(Color("green"),46))

df_tot["color"] =  colors
df_tot.head()

,Employee,betweennes,centrality_rank,rank,color
0,CEO,1.000000,1.000000,1.000000,red
1,Production.mgr1,0.783388,0.308151,0.545770,#fc0b00
7,Sales.Dir,0.255296,0.775986,0.515641,#f91600
6,Sales.mgr1,0.290588,0.532339,0.411464,#f72100
3,Admin.Dir,0.342256,0.395253,0.368755,#f42b00


In [ ]:
true_color = []
for i in colors:
    true_color.append(str(i))

df_tot["color"] =  true_color

In [15]:
num = 0
for node in G.nodes(data=True):
    size = (df_tot[df_tot["Employee"]==node[0]]["rank"].iloc[0]*10)+5
    G.add_node(node[0], size = 10, colore="default")
    num +=1
for node in G.nodes(data=True):
    print(node)


for u, v in G.edges():
    if data[(data["Person 1"]==u)&(data["Person 2"]==v)]["weight"].empty:
        weight = float(data[(data["Person 2"]==u)&(data["Person 1"]==v)]["weight"])
    else:
        weight = float(data[(data["Person 1"]==u)&(data["Person 2"]==v)]["weight"])
    G[u][v]["color"] = "black"

net = Network(notebook=True, cdn_resources='remote', height="1000")
net.from_nx(G, show_edge_weights=False)
net.show("example.html");

('Admin.mgr1', {'size': 10, 'colore': 'default'})
('Admin.2', {'size': 10, 'colore': 'default'})
('Sales.mgr3', {'size': 10, 'colore': 'default'})
('Sales.8', {'size': 10, 'colore': 'default'})
('Production.2', {'size': 10, 'colore': 'default'})
('Production.3', {'size': 10, 'colore': 'default'})
('Production.7', {'size': 10, 'colore': 'default'})
('Sales.1', {'size': 10, 'colore': 'default'})
('Sales.2', {'size': 10, 'colore': 'default'})
('Admin.1', {'size': 10, 'colore': 'default'})
('Admin.5', {'size': 10, 'colore': 'default'})
('Sales.7', {'size': 10, 'colore': 'default'})
('Sales.9', {'size': 10, 'colore': 'default'})
('Production.Dir', {'size': 10, 'colore': 'default'})
('Production.mgr3', {'size': 10, 'colore': 'default'})
('Sales.3', {'size': 10, 'colore': 'default'})
('Sales.mgr2', {'size': 10, 'colore': 'default'})
('Sales.4', {'size': 10, 'colore': 'default'})
('Sales.5', {'size': 10, 'colore': 'default'})
('Sales.6', {'size': 10, 'colore': 'default'})
('Sales.mgr1', {'size

C:\Users\teuta\AppData\Local\Temp\ipykernel_16576\21087983.py:14: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

C:\Users\teuta\AppData\Local\Temp\ipykernel_16576\21087983.py:12: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



example.html
